<a href="https://colab.research.google.com/github/Jwavely/Google-Colaboratory/blob/main/boston_housing_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import boston_housing

In [2]:
(train_data, train_targets),(test_data, test_targets) = boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [7]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [9]:
print(train_targets.shape)
print(test_targets.shape)

(404,)
(102,)


In [10]:
mean = train_data.mean(axis = 0)
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std
test_data -= mean
test_data /= std

테스트 데이터를 정규화할 때 사용한 값이 훈련 데이터에서 계산한 값임을 주목하세요. 머신 러닝 작업 과정에서 절대로 테스트 데이터에서 계산한 어떤 값도 사용해서는 안 됩니다. 데이터 정규화처럼 간단한 작업조차도 그렇습니다.


# **모델 구성**

In [14]:
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu',
                         input_shape = (train_data.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer = 'rmsprop',loss = 'mse',metrics = ['mae'])
  return model

이 네트워크의 마지막 층은 하나의 유닛을 가지고 있고 활성화 함수가 없습니다(선형 층이라고 부릅니다). 이것이 전형적인 스칼라 회귀(하나의 연속적인 값을 예측하는 회귀)를 위한 구성입니다. 활성화 함수를 적용하면 출력 값의 범위를 제한하게 됩니다. 예를 들어 마지막 층에 sigmoid 활성화 함수를 적용하면 네트워크가 0과 1 사이의 값을 예측하도록 학습될 것입니다. 여기서는 마지막 층이 순수한 선형이므로 네트워크가 어떤 범위의 값이라도 예측하도록 자유롭게 학습됩니다.

이 모델은 mse 손실 함수를 사용하여 컴파일합니다. 이 함수는 평균 제곱 오차mean squared error의 약어로 예측과 타깃 사이 거리의 제곱입니다. 회귀 문제에서 널리 사용되는 손실 함수입니다.

훈련하는 동안 모니터링을 위해 새로운 지표인 평균 절대 오차Mean Absolute Error, MAE를 측정합니다. 이는 예측과 타깃 사이 거리의 절댓값입니다. 예를 들어 이 예제에서 MAE가 0.5면 예측이 평균적으로 500달러 정도 차이가 난다는 뜻입니다.

## **K-fold cross-validation**

In [19]:
import numpy as np

k = 4

num_val_samples = len(train_data) //4
num_epochs = 100
all_scores = []
for i in range(k):
  print('처리중인 폴드 #',i)
  val_data = train_data[i *num_val_samples:(i+1)*num_val_samples] # 검정 데이터 준비
  val_targets = train_targets[i *num_val_samples:(i+1)*num_val_samples] # 검정 데이터 준비

  partial_train_data = np.concatenate(
      [train_data[:i*num_val_samples],
       train_data[(i+1)*num_val_samples:]],
       axis = 0)
  partial_train_targets = np.concatenate(
      [train_targets[:i*num_val_samples],
       train_targets[(i+1)*num_val_samples:]],
       axis = 0)

  model =build_model()
  model.fit(partial_train_data, partial_train_targets, epochs=num_epochs,batch_size=1, verbose=0)
  val_mse,val_mae = model.evaluate(val_data, val_targets,verbose=0)
  all_scores.append(val_mae)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [20]:
all_scores

[2.0603389739990234,
 2.6399588584899902,
 2.8592891693115234,
 2.4422354698181152]

In [21]:
np.mean(all_scores)

2.500455617904663

In [22]:
model = build_model()  # 새롭게 컴파일된 모델을 얻습니다.
model.fit(train_data, train_targets,  # 전체 데이터로 훈련시킵니다.
          epochs=80, batch_size=16, verbose=0)

In [23]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

4/4 [==============================] - 0s 4ms/step - loss: 16.8123 - mae: 2.5967


In [24]:
>>> test_mae_score

2.5966858863830566